# FLAG EXPLORATION

In [141]:
import pandas as pd
import numpy as np
import PIL
import PIL.ImageShow
import PIL.Image
import os
import matplotlib

### LOADING ISO NAMES FILE

In [3]:
iso_names = pd.read_csv(os.path.join('data', 'country_iso2_names.csv'))

print(iso_names.iloc[10])

Name    Armenia
Code         AM
Name: 10, dtype: object


### LOADING FLAG

In [229]:
def get_png_name(name, iso_names):
    return os.path.join('data', 'w2560', '{}.png'.format(iso_names.loc[iso_names.Name == name].Code.item().lower()))

def resize_image(img: PIL.Image.Image):

    classic_size = [2560, 1707]

    img_size = img.size

    if img_size[1] > classic_size[1]:
        div_factor = img_size[1] / classic_size[1]
        new_x = int(np.round(img_size[0] / div_factor))
        left = (classic_size[0] - new_x) // 2

        resized_img = PIL.Image.new('RGBA', classic_size, (0, 0, 0, 0))
        resized_img.paste(img.resize((new_x, classic_size[1])), (left, 0))

    elif img_size[1] < classic_size[1]:
        top = (classic_size[1] - img_size[1]) // 2

        resized_img = PIL.Image.new('RGBA', classic_size, (0, 0, 0, 0))
        resized_img.paste(img, (0, top))
    
    else:
        return img.convert(mode='RGBA')

    return resized_img

def display_image(img_path):

    image = PIL.Image.open(img_path)
    resized_img = resize_image(image)

    PIL.ImageShow.show(resized_img)

def get_flag_img(country_name):
    image = PIL.Image.open(get_png_name(country_name, iso_names))
    resized_img = resize_image(image)
    return resized_img

def display_country_flag(country_name):
    display_image(get_png_name(country_name, iso_names))

In [36]:
display_country_flag('Nepal')

In [109]:
img_1 = get_flag_img('France')
img_2 = get_flag_img('Italy')
flag_1 = np.array(img_1)
flag_2 = np.array(img_2)

In [222]:
def get_processed_flag(flag):
    hsv_flag = matplotlib.colors.rgb_to_hsv(flag[:,:,:3] / 255)
    processed_flag = np.zeros(list(hsv_flag.shape[:2])+[3], dtype=np.uint8)

    (h,s,v) = (0,1,2)    

    # Hue
    processed_flag[:,:,h] = hsv_flag[:,:,h] * 10

    # Saturation
    processed_flag[:,:,s] = hsv_flag[:,:,s] * 0

    # Value
    processed_flag[:,:,v] = hsv_flag[:,:,v] * 1



    np.round(processed_flag)
    return processed_flag
    

def get_flag_mask(flag_1, flag_2):

    processed_flag_1 = get_processed_flag(flag_1)
    processed_flag_2 = get_processed_flag(flag_2)

    classic_size = [1707, 2560]
    flag_mask = np.where((processed_flag_1 == processed_flag_2).all(axis=2), np.ones(classic_size, dtype=np.uint8), np.zeros(classic_size, dtype=np.uint8))
    return flag_mask

In [162]:
flag_mask = get_flag_mask(flag_1, flag_2)

0.6617144807603607
0.6190690729966025


In [136]:
print(flag_1.shape)
classic_size = [1707, 2560]
flag_mask = np.where((flag_1 == flag_2).all(axis=2), np.ones(classic_size, dtype=np.uint8), np.zeros(classic_size, dtype=np.uint8))
print(flag_mask.shape)

(1707, 2560, 4)
(1707, 2560)


In [137]:
flag_mask_rgb = np.stack([flag_mask*255]*3, axis=2)
print(flag_mask_rgb.shape)

(1707, 2560, 3)


In [138]:
im = PIL.Image.fromarray(flag_mask_rgb, 'RGB')
im.show()

In [215]:
def get_union_flag(country_name_true, country_name_guess):

    flag_true = np.array(get_flag_img(country_name_true))
    flag_guess = np.array(get_flag_img(country_name_guess))

    flag_mask = get_flag_mask(flag_true, flag_guess)
    flag_mask_rgb = np.stack([flag_mask]*4, axis=2)

    union_flag = np.where(flag_mask_rgb==1, flag_true, np.zeros_like(flag_true))
    return union_flag

def combine_union_flags(union_original, union_to_combine):
    new_union = union_original.copy()
    new_union[union_to_combine[:,:,3] != 0] = union_to_combine[union_to_combine[:,:,3] != 0]
    return new_union

def show_union_flag(union_flag):
    im = PIL.Image.fromarray(union_flag, 'RGBA')
    im.show()

In [234]:
ir_fr_union_flag = get_union_flag('Ireland', 'France')
show_union_flag(ir_fr_union_flag)

In [235]:
ir_it_union_flag = get_union_flag('Ireland', 'Italy')
combined_flag = combine_union_flags(ir_fr_union_flag, ir_it_union_flag)
show_union_flag(combined_flag)

In [232]:
ir_sl_union_flag = get_union_flag('Ireland', 'Sri Lanka')
combined_flag = combine_union_flags(combined_flag, ir_sl_union_flag)
show_union_flag(combined_flag)